# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

We begin with mounting the Google Drive in order to fetch the training and validation data from the location

In [1]:
import numpy as np
import os
import datetime
import os

In [2]:
from PIL import Image
import cv2 as cv
from cv2 import imread
#from scipy.misc import imread, imresize
import warnings
warnings.filterwarnings("ignore")

We set the random seed so that the results don't vary drastically.

In [3]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [4]:
# train_doc = np.random.permutation(open('/content/drive/MyDrive/ML_C47_Gesture_Recognition/Project_data/train.csv').readlines())
# val_doc = np.random.permutation(open('/content/drive/MyDrive/ML_C47_Gesture_Recognition/Project_data/val.csv').readlines())
train_doc = np.random.permutation(open('datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('datasets/Project_data/val.csv').readlines())
batch_size = 50

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [17]:
# selecting alternate frames from 2 to 30.
img_seq_idx = range(2,30,2)
# image dimensions
val_y, val_z = 120, 120

In [18]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = img_seq_idx
    x = len(img_idx)
    y, z = val_y, val_z
    while True:
        t = np.random.permutation(folder_list)
#         if (len(t)%batch_size) == 0:
#             num_batches = int(len(t)/batch_size)
#         else:
        num_batches = len(t)//batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x, y, z, 3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                    #crop the images and resize them. Note that the images are of 2 different shape
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    # One way of implementing this is turning the images into squares and resizing them
                    longer_side = image.shape[1] if  image.shape[1] > image.shape[0] else image.shape[0];
                    shorter_side = image.shape[1] if  image.shape[1] < image.shape[0] else image.shape[0];
                    diff_px = longer_side - shorter_side
                    crop_start = diff_px//2
                    crop_end = crop_start + shorter_side
                    image = image[:, crop_start:crop_end]

                    resized_im = cv.resize(image, dsize=(y,z))


                    batch_data[folder,idx,:,:,0] = resized_im[:,:,0]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = resized_im[:,:,1]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = resized_im[:,:,2]/255 #normalise and feed in the image

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


        # write the code for the remaining data points which are left after full batches


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [47]:
curr_dt_time = datetime.datetime.now()
# train_path = '/content/drive/MyDrive/ML_C47_Gesture_Recognition/Project_data/train'
# val_path = '/content/drive/MyDrive/ML_C47_Gesture_Recognition/Project_data/val'
train_path = 'datasets/Project_data/train'
val_path = 'datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 50# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 50


## Model Building
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

Importing necessary librarues and setting all the values

In [55]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout, GlobalAveragePooling2D, GlobalAveragePooling3D, ConvLSTM2D
from tensorflow.keras.layers import Dropout
from keras.layers.convolutional import Conv2D, Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

#input shape for the st layer
input_shape = (len(img_seq_idx), val_y, val_z, 3)
np.random.seed(30)

# Model 01

In [21]:
# model1 = Sequential()
# model1.add(Conv3D(16, kernel_size=3, activation='relu', input_shape=input_shape))
# model1.add(MaxPooling3D(pool_size=2))

# model1.add(Conv3D(32, kernel_size=3, activation='relu'))
# model1.add(MaxPooling3D(pool_size=2))

# model1.add(Flatten())
# model1.add(Dense(128, activation='relu'))
# model1.add(Dense(5, activation='softmax'))

## Model 02

In [28]:
# model2 = Sequential()

# model2.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape))
# model2.add(Conv3D(64, kernel_size=3, activation='relu'))
# model2.add(MaxPooling3D(pool_size=(2, 2, 2)))
# model2.add(BatchNormalization())

# model2.add(Conv3D(128, kernel_size=3, activation='relu'))
# model2.add(MaxPooling3D(pool_size=(1, 2, 2)))
# model2.add(BatchNormalization())

# model2.add(Conv3D(256, kernel_size=(1, 3, 3), activation='relu'))
# model2.add(MaxPooling3D(pool_size=(1, 2, 2)))
# model2.add(BatchNormalization())

# model2.add(Conv3D(512, kernel_size=(1, 3, 3), activation='relu'))
# model2.add(Conv3D(512, kernel_size=(1, 3, 3), activation='relu'))
# model2.add(MaxPooling3D(pool_size=(1, 2, 2)))
# model2.add(BatchNormalization())

# model2.add(Flatten())
# model2.add(Dense(512, activation='relu'))
# model2.add(BatchNormalization())
# model2.add(Dense(5, activation='softmax'))

## Model 03

In [35]:


# model3 = Sequential()

# model3.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape))
# model3.add(Conv3D(64, kernel_size=3, activation='relu'))
# model3.add(MaxPooling3D(pool_size=(2, 2, 2)))
# model3.add(BatchNormalization())
# model3.add(Dropout(0.2))

# model3.add(Conv3D(128, kernel_size=3, activation='relu'))
# model3.add(MaxPooling3D(pool_size=(1, 2, 2)))
# model3.add(BatchNormalization())
# model3.add(Dropout(0.2))

# model3.add(Conv3D(256, kernel_size=(1, 3, 3), activation='relu'))
# model3.add(MaxPooling3D(pool_size=(1, 2, 2)))
# model3.add(BatchNormalization())
# model3.add(Dropout(0.2))

# model3.add(Flatten())
# model3.add(Dense(512, activation='relu'))
# model3.add(BatchNormalization())
# model3.add(Dense(5, activation='softmax'))

## Model 04

In [42]:
# model4 = Sequential()

# model4.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape))
# model4.add(Conv3D(64, kernel_size=3, activation='relu'))
# model4.add(MaxPooling3D(pool_size=(2, 2, 2)))
# model4.add(BatchNormalization())
# model4.add(Dropout(0.2))

# model4.add(Conv3D(128, kernel_size=3, activation='relu'))
# model4.add(MaxPooling3D(pool_size=(1, 2, 2)))
# model4.add(BatchNormalization())
# model4.add(Dropout(0.2))

# model4.add(Conv3D(256, kernel_size=(1, 3, 3), activation='relu'))
# model4.add(MaxPooling3D(pool_size=(1, 2, 2)))
# model4.add(BatchNormalization())
# model4.add(Dropout(0.2))

# model4.add(GlobalAveragePooling3D())
# model4.add(Dense(512, activation='relu'))
# model4.add(BatchNormalization())
# model4.add(Dense(5, activation='softmax'))


## Model 05

In [56]:
model5 = Sequential()
model5.add(TimeDistributed(
    Conv2D(8, (3,3), activation='relu'), input_shape=input_shape)
)
model5.add(BatchNormalization())
model5.add(TimeDistributed(
    Conv2D(16, (3,3), activation='relu'))
)
model5.add(BatchNormalization())
model5.add(
    ConvLSTM2D(8, kernel_size = 3, return_sequences=False)
)
model5.add(BatchNormalization())
model5.add(TimeDistributed(
    Dense(64, activation='relu'))
)
model5.add(BatchNormalization())
model5.add(GlobalAveragePooling2D())
model5.add(Dense(64, activation='relu'))
model5.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [57]:
model = model5 #Reference the model being created
optimiser = tf.keras.optimizers.Adam(lr=0.01) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_5 (TimeDis  (None, 14, 118, 118, 8)  224       
 tributed)                                                       
                                                                 
 batch_normalization_20 (Bat  (None, 14, 118, 118, 8)  32        
 chNormalization)                                                
                                                                 
 time_distributed_6 (TimeDis  (None, 14, 116, 116, 16)  1168     
 tributed)                                                       
                                                                 
 batch_normalization_21 (Bat  (None, 14, 116, 116, 16)  64       
 chNormalization)                                                
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 114, 114, 8)      

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [58]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [59]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', save_freq='epoch')

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [60]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [61]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  datasets/Project_data/train ; batch size = 50
Epoch 1/50
14/14 [==============================] - ETA: 0s - loss: 1.4043 - categorical_accuracy: 0.3443Source path =  datasets/Project_data/val ; batch size = 50

Epoch 00001: saving model to model_init_2023-06-2917_42_04.117924/model-00001-1.40431-0.34429-2.32928-0.23000.h5
14/14 [==============================] - 26s 2s/step - loss: 1.4043 - categorical_accuracy: 0.3443 - val_loss: 2.3293 - val_categorical_accuracy: 0.2300 - lr: 0.0100
Epoch 2/50
14/14 [==============================] - ETA: 0s - loss: 1.0541 - categorical_accuracy: 0.5171
Epoch 00002: saving model to model_init_2023-06-2917_42_04.117924/model-00002-1.05415-0.51714-2.65623-0.25000.h5
14/14 [==============================] - 23s 2s/step - loss: 1.0541 - categorical_accuracy: 0.5171 - val_loss: 2.6562 - val_categorical_accuracy: 0.2500 - lr: 0.0100
Epoch 3/50
14/14 [==============================] - ETA: 0s - loss: 0.9594 - categorical_accuracy: 0.5729
Epoc